In [41]:
import numpy as np
import random
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, clone_model
from keras.layers import Dense
from keras.optimizers import Adam

# Veri setini yükleme ve ön işleme
data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Basit bir sinir ağı modeli oluşturma
def create_model():
    model = Sequential([
        Dense(20, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(10, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(), loss='mean_squared_error')
    return model

# Uygunluk fonksiyonu
def fitness(model, X, y):
    score = model.evaluate(X, y, verbose=0)
    return score

# Seçim: En iyi bireyleri seçme
def select_parents(population, fitness_scores, num_parents=4):
    parents_idx = np.argsort(fitness_scores)[:num_parents]
    return [population[i] for i in parents_idx]

# Çaprazlama
def crossover(parent_1, parent_2):
    child = clone_model(parent_1)
    child.set_weights(parent_1.get_weights())

    parent_2_weights = parent_2.get_weights()
    for i in range(len(child.get_weights())):
        if np.random.uniform(0, 1) > 0.5:
            child.get_weights()[i] = parent_2_weights[i]

    return child

# Mutasyon
def mutate(child, mutation_rate=0.1):
    child_weights = child.get_weights()
    for i in range(len(child_weights)):
        if np.random.uniform(0, 1) < mutation_rate:
            mutation_value = np.random.normal(loc=0.0, scale=1.0, size=child_weights[i].shape)
            child_weights[i] += mutation_value
    child.set_weights(child_weights)
    return child

# Model parametrelerini ve sonuçlarını kaydetmek için DataFrame
results_df = pd.DataFrame(columns=["Generation", "ModelID", "FitnessScore", "ModelWeights"])

# Popülasyonu başlatma
population_size = 10
population = [create_model() for _ in range(population_size)]

# Optimizasyon döngüsü
generations = 50
for generation in range(generations):
    fitness_scores = [fitness(model, X_train, y_train) for model in population]

    # Sonuçları kaydetme
    for i, model in enumerate(population):
        model_id = f"Gen_{generation}_Model_{i}"
        new_row = pd.DataFrame({
            "Generation": [generation],
            "ModelID": [model_id],
            "FitnessScore": [fitness_scores[i]],
            "ModelWeights": [model.get_weights()]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

    parents = select_parents(population, fitness_scores)
    next_generation = parents.copy()

    # Yeni nesil oluşturma
    while len(next_generation) < population_size:
        parent_1, parent_2 = random.sample(parents, 2)
        child = crossover(parent_1, parent_2)
        child = mutate(child)
        next_generation.append(child)

    population = next_generation
    print(f"Generation {generation + 1}/{generations} Complete")

# Sonuçları kaydedilen DataFrame'i yazdırma
print(results_df.head())

# En iyi modeli bulma ve test etme
best_model = min(population, key=lambda model: fitness(model, X_train, y_train))
test_score = fitness(best_model, X_test, y_test)
print("Test Score of the Best Model:", test_score)


/var/folders/w9/3z_zbj956tj44glgzg44rgbr0000gn/T/ipykernel_35033/678433953.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


Generation 1/50 Complete


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.